In [ ]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:10 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:12 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Fetched 108 kB in 7s (15.7 kB/s)
Reading package lists... Done


In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-02-06 18:49:56--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.3’

postgresql-42.2.16. 100%[===================>] 979.38K  5.54MB/s    in 0.2s    

2023-02-06 18:49:57 (5.54 MB/s) - ‘postgresql-42.2.16.jar.3’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

### Create DataFrames to match tables

In [ ]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [ ]:
# Create the customers_table DataFrame
customers_df = df.groupby("customer_id").agg({'customer_id':'count'}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|    1567510|             1|
|   19502021|             1|
|   12819130|             1|
|   35329257|             2|
|     108460|             1|
|    5453476|             1|
|   29913055|             1|
|   30717305|             1|
|    1570030|             1|
|   19032020|             1|
|   44178035|             1|
|   26079415|             2|
|   14230926|             1|
|   43478048|             2|
|   43694941|             1|
|   12318815|             3|
|   13731855|             1|
|     740134|             1|
|   41956754|             1|
|   20324070|             3|
+-----------+--------------+
only showing top 20 rows



In [ ]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(['product_id','product_title'])
products_df = products_df.drop_duplicates()



In [ ]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(['review_id','customer_id','product_parent',to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+--------------+-----------+
|     review_id|customer_id|product_parent|review_date|
+--------------+-----------+--------------+-----------+
|R3O9SGZBVQBV76|    3653882|     937001370| 2015-08-31|
| RKH8BNC3L5DLF|   14661224|     484010722| 2015-08-31|
|R2HLE8WKZSU3NL|   27324930|     361166390| 2015-08-31|
|R31U3UH5AZ42LL|    7211452|     958035625| 2015-08-31|
|R2SV659OUJ945Y|   12733322|     765328221| 2015-08-31|
| RA51CP8TR5A2L|    6576411|     230493695| 2015-08-31|
| RB2Q7DLDN6TH6|   11811565|     549298279| 2015-08-31|
|R2RHFJV0UYBK3Y|   49401598|     844009113| 2015-08-31|
|R2Z6JOQ94LFHEP|   45925069|     263720892| 2015-08-31|
| RX27XIIWY5JPB|   44751341|     124278407| 2015-08-31|
+--------------+-----------+--------------+-----------+
only showing top 10 rows



In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id','star_rating','helpful_votes','total_votes','vine','verified_purchase']).drop_duplicates()
vine_df.show(10)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| R6SP85SIT1IWP|          5|            0|          0|   N|                Y|
|R1MQHKJV0ZV557|          5|            0|          0|   N|                Y|
|R2NUIAT3WBD8SA|          5|            0|          0|   N|                Y|
|R1SA2XY18AYOYY|          5|            0|          0|   N|                Y|
|R1PNURL4UZ0KZL|          4|            0|          1|   N|                Y|
|R3UYMMI8APFV4D|          4|            0|          0|   N|                Y|
| RRFFGVG3SUEG5|          5|            0|          0|   N|                N|
|R20NYA5V0UF9NE|          5|           27|         28|   N|                Y|
|R2U6KL3JZFTX8L|          5|            0|          0|   N|                Y|
|R2LBVP7MVALFOO|          5|            0|          1|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')
mode = "append"
jdbc_url="jdbc:postgresql://dbchallenge.czbhd3a9se33.us-east-1.rds.amazonaws.com:5432/challenge17_db"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Enter database password··········


In [ ]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [ ]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)